In [19]:
import yfinance as yf

token = 'BTC-USD'
# start_date = '2015-01-01'

data = yf.download(token, interval='1d', period='max')
data = data.droplevel('Ticker', axis=1)
data.to_csv('data/BTC-USD.csv')
data

C:\Users\ivoto\AppData\Local\Temp\ipykernel_6244\343183753.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(token, interval='1d', period='max')
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Date,,,,,
2014-09-17,457.334015,468.174011,452.421997,465.864014,21056800
2014-09-18,424.440002,456.859985,413.104004,456.859985,34483200
2014-09-19,394.795990,427.834991,384.532013,424.102997,37919700
2014-09-20,408.903992,423.295990,389.882996,394.673004,36863600
2014-09-21,398.821014,412.425995,393.181000,408.084991,26580100
...,...,...,...,...,...
2025-07-30,117831.187500,118780.726562,115800.828125,117921.992188,68896148592
2025-07-31,115758.203125,118919.984375,115505.218750,117833.632812,69370346018
2025-08-01,113320.085938,116060.773438,112724.445312,115738.953125,91294530181


In [20]:
import pandas as pd
from ta.trend import EMAIndicator, MACD
from ta.momentum import RSIIndicator

# Carga tus datos en 'data' con columna 'Close'
data = pd.read_csv('data/BTC-USD.csv')

# EMAs
data['ema20'] = EMAIndicator(data['Close'], window=20).ema_indicator()
data['ema50'] = EMAIndicator(data['Close'], window=50).ema_indicator()

# RSIs
data['rsi10']  = RSIIndicator(data['Close'], window=10).rsi()
data['rsi30']  = RSIIndicator(data['Close'], window=30).rsi()
data['rsi200'] = RSIIndicator(data['Close'], window=200).rsi()

# MACD
macd_ind = MACD(data['Close'], window_fast=12, window_slow=26, window_sign=9)
data['macd']   = macd_ind.macd()
data['signal'] = macd_ind.macd_signal()
data['hist']   = macd_ind.macd_diff()

# Señal de subida de precio
data['Price-Up'] = (data['Close'].shift(-1) > data['Close']).astype(int)

# Limpieza y exportación
data.dropna(inplace=True)
data.to_csv('data/BTC-I.csv', index=False)
data
print(data.tail())


            Date          Close           High            Low           Open  \
3969  2025-07-30  117831.187500  118780.726562  115800.828125  117921.992188   
3970  2025-07-31  115758.203125  118919.984375  115505.218750  117833.632812   
3971  2025-08-01  113320.085938  116060.773438  112724.445312  115738.953125   
3972  2025-08-02  112526.914062  114021.601562  112005.765625  113320.390625   
3973  2025-08-04  115197.789062  115698.125000  114152.195312  114217.476562   

           Volume          ema20          ema50      rsi10      rsi30  \
3969  68896148592  116951.235985  112751.264954  54.582307  59.345320   
3970  69370346018  116837.613808  112869.184098  43.415070  55.853819   
3971  91294530181  116502.611153  112886.866523  34.256080  52.122739   
3972  56870866000  116123.973335  112872.750740  31.828916  50.976726   
3973  55234772992  116035.765309  112963.928714  46.113957  54.464304   

         rsi200         macd       signal         hist  Price-Up  
3969  55.2804

# Stream_Crypto_Data 

In [7]:
import json 
import websocket
import pandas as pd
import numpy as np

coin = 'btcusdt@kline_1m'
socket = 'wss://stream.binance.com:9443/stream?streams=' + coin
print(socket)


ModuleNotFoundError: No module named 'websocket'

In [ ]:
def extract_ohlcv(source):
    event_time = pd.to_datetime(source['data']['E'], unit='ms')
    open = source['data']['k']['o']
    high = source['data']['k']['h']
    low = source['data']['k']['l']
    close = source['data']['k']['c']   
    volume = source['data']['k']['v']

    data = {'Open': open, 'High': high, 'Low': low, 'Close': close, 'Volume': volume}

    df = pd.DataFrame(data, index=[event_time])
    df.index.name = 'timestamp'
    df = df.astype(float)
    df = df.reset_index()
    print(df)
    return df

def on_message(ws, message):
    message = json.loads(message)
    extract_ohlcv(message)

In [ ]:
ws = websocket.WebSocketApp(socket, on_message=on_message)
ws.run_forever()

                timestamp      Open     High       Low     Close    Volume
0 2025-02-27 17:53:18.014  84573.53  84593.7  84494.26  84494.26  10.41565
                timestamp      Open     High       Low     Close    Volume
0 2025-02-27 17:53:20.015  84573.53  84593.7  84494.26  84494.27  10.43499
                timestamp      Open     High       Low     Close    Volume
0 2025-02-27 17:53:22.029  84573.53  84593.7  84494.26  84494.27  10.61498


True